In [ ]:
import warnings
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from autumn.core.inputs.tb_camau import queries
from autumn.core.inputs import get_death_rates_by_agegroup, get_life_expectancy_by_agegroup, get_crude_birth_rate
import pathlib

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
age_string_map = {
    0: "0-4",
    5: "5-14",
    15: "15-34",
    35: "35-49",
    50: "50+",
}
age_breakpoints = [0,5,15,35,50]

In [ ]:
def get_death_rates_by_year(year):
    death_rates_from_db =get_death_rates_by_agegroup(age_breakpoints, "VNM")
    df = pd.DataFrame(death_rates_from_db[0], index=death_rates_from_db[1])
    if year not in death_rates_from_db[1]:
        print("No data for this year!")
    else:
        return(df.loc[year]) 

In [ ]:
death_series = get_death_rates_by_year(1952.5)
death_series

In [ ]:
init_pop = 100000

In [ ]:
df = pd.DataFrame({
    'age': death_series.index,
    'qx': death_series.values
})

# Initialize the 'lx' column with an initial population of 100,000 for age 0
df['lx'] = 100000

# Calculate population 'lx' for each age group
for i in range(1, len(df)):
    df.loc[i, 'lx'] = df.loc[i - 1, 'lx'] - (df.loc[i - 1, 'qx'] * df.loc[i - 1, 'lx'])

#calculate number of death
df["dx"] = 0
for x in range(5):
    if x >=4:
        df["dx"][x] = df["lx"][x]
    else:
        df['dx'] =  df['lx'] * df['qx']

#μx: is the force of mortality, i.e., represents the instantaneous rate at which people are dying.
df["μx"] = 0
df["μx"][0] = 2.0692602739726 * df["dx"][0] /(2*df["lx"][0])
for x in range(1,5):
    df["μx"][x] = (df["dx"][x-1]+df["dx"][x])/(2*df["lx"][x])
#Tx: is the total years of life to be lived by those aged exactly x (not the random variable Tx) until they all die
df["Tx"] = 0
for x in range(5):
    df["Tx"][x] = 0
    for y in range(x,5):
        df["Tx"][x] += df["lx"][y]
df["e0x"] = df["Tx"]/df["lx"] - 0.5

df["Lx"] = 0.000
for x in range(5):
    df["Lx"][x] = df["Tx"][x]-df["lx"][x]
df